# <center>VAE-baseline for SVHN </center>

## Load Data

In [1]:
!pip3 install --upgrade torch torchvision

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (0.4.0)
Requirement already up-to-date: torchvision in /usr/local/lib/python3.6/dist-packages (0.2.1)
Requirement not upgraded as not directly required: pillow>=4.1.1 in /usr/local/lib/python3.6/dist-packages (from torchvision) (5.1.0)
Requirement not upgraded as not directly required: six in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.11.0)
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.6/dist-packages (from torchvision) (1.14.3)


In [0]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from IPython.display import Image
from google.colab import files


In [3]:
#Set random seed 
torch.manual_seed(512)

In [4]:
#Get the SVHN train images 
svhn = datasets.SVHN('./data/svhn/', download = True)

Using downloaded and verified file: ./data/svhn/train_32x32.mat


In [0]:
# Organize training data in batches, 
# normalize them to have values between [-1, 1] (?)

train_images = torch.utils.data.DataLoader ( datasets.SVHN('./data/svhn/', download=False,
                               transform=transforms.Compose([
                               #transforms.Resize(64), 
                               #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                               transforms.ToTensor(),])) , 
                               batch_size = 128, shuffle = True)

## Model

We will use the arquitecture suggested by [Radford et al](https://arxiv.org/abs/1511.06434) for both the encoder and decoder. With convolutional layers in the encoder and fractionally-strided  convolutions  in  the  decoder.   In  each convolutional layer in the encoder we double the number of filters present in the previous layer and use a convolutional stride of 2.  In each convolutional layer in the decoder we use a fractional stride of 2 and halve the number of filters on each layer.

In [0]:
class VAE(nn.Module):

    def __init__(self, image_size ,  hidden_dim , encoding_dim):
        
        super(VAE, self).__init__()
        
        self.encoding_dim = encoding_dim
        self.image_size = image_size
        self.hidden_dim = hidden_dim 
        
        # Decoder - Fractional strided convolutional layers
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, 1, 0, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1, bias = False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 3, 4, 2, 1, bias = False),
            nn.Sigmoid() # nn.Tanh()  
        )
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(32, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 0, bias = False),
            nn.Sigmoid()
        )
        
        # Fully-connected layers
        self.fc1 = nn.Linear(256, self.hidden_dim)
        self.fc21 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc22 = nn.Linear(self.hidden_dim, self.encoding_dim)
        self.fc3 = nn.Linear(self.encoding_dim, self.hidden_dim)
        self.fc4 = nn.Linear(self.hidden_dim, 256)
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        h4 = F.sigmoid(self.fc4(h3))
        return self.decoder( h4.view(z.size(0),-1,1,1) ) 

        
    def forward(self, x):
        
        # Encode 
        encoded = F.relu(self.fc1( self.encoder(x).view(x.size(0), -1) ) )
        
        #Obtain mu and logvar
        mu = self.fc21( encoded )
        logvar = self.fc22 ( encoded )
        
        #Reparametrization trick
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = eps.mul(std).add_(mu)
        
        # Decode 
        decoded = self.decode(z)

        # return decoded, mu, logvar
        return decoded, mu , logvar

    
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [0]:
#Define model
model = VAE(32, 128, 32).cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3, amsgrad=True)

#Train model
def train(epoch):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_images):
        data = Variable(data).cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_images.dataset),
                100. * batch_idx / len(train_images),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_images.dataset)))

In [8]:
num_epochs = 100
for epoch in range(1, num_epochs):
    train(epoch)

Train Epoch: 1 [0/73257 (0%)]	Loss: 2135.716309
Train Epoch: 1 [6400/73257 (9%)]	Loss: 2051.954590
Train Epoch: 1 [12800/73257 (17%)]	Loss: 2045.068848
Train Epoch: 1 [19200/73257 (26%)]	Loss: 2028.473145
Train Epoch: 1 [25600/73257 (35%)]	Loss: 2035.691895
Train Epoch: 1 [32000/73257 (44%)]	Loss: 2041.484009
Train Epoch: 1 [38400/73257 (52%)]	Loss: 1994.399658
Train Epoch: 1 [44800/73257 (61%)]	Loss: 1999.957642
Train Epoch: 1 [51200/73257 (70%)]	Loss: 1989.978271
Train Epoch: 1 [57600/73257 (79%)]	Loss: 1972.124878
Train Epoch: 1 [64000/73257 (87%)]	Loss: 1953.391968
Train Epoch: 1 [70400/73257 (96%)]	Loss: 1994.351074
====> Epoch: 1 Average loss: 2015.1637
Train Epoch: 2 [0/73257 (0%)]	Loss: 1951.367065
Train Epoch: 2 [6400/73257 (9%)]	Loss: 1976.494141
Train Epoch: 2 [12800/73257 (17%)]	Loss: 1929.584961
Train Epoch: 2 [19200/73257 (26%)]	Loss: 1970.116943
Train Epoch: 2 [25600/73257 (35%)]	Loss: 1985.125122
Train Epoch: 2 [32000/73257 (44%)]	Loss: 1987.565796
Train Epoch: 2 [38400

Train Epoch: 5 [70400/73257 (96%)]	Loss: 1927.371582
====> Epoch: 5 Average loss: 1936.9043
Train Epoch: 6 [0/73257 (0%)]	Loss: 1922.021973
Train Epoch: 6 [6400/73257 (9%)]	Loss: 1940.452881
Train Epoch: 6 [12800/73257 (17%)]	Loss: 1948.143066
Train Epoch: 6 [19200/73257 (26%)]	Loss: 1933.294067
Train Epoch: 6 [25600/73257 (35%)]	Loss: 1968.636963
Train Epoch: 6 [32000/73257 (44%)]	Loss: 1949.649048
Train Epoch: 6 [38400/73257 (52%)]	Loss: 1928.587158
Train Epoch: 6 [44800/73257 (61%)]	Loss: 1954.813721
Train Epoch: 6 [51200/73257 (70%)]	Loss: 1938.494141
Train Epoch: 6 [57600/73257 (79%)]	Loss: 1936.938843
Train Epoch: 6 [64000/73257 (87%)]	Loss: 1917.013306
Train Epoch: 6 [70400/73257 (96%)]	Loss: 1933.482788
====> Epoch: 6 Average loss: 1934.7572
Train Epoch: 7 [0/73257 (0%)]	Loss: 1912.234009
Train Epoch: 7 [6400/73257 (9%)]	Loss: 1949.760620
Train Epoch: 7 [12800/73257 (17%)]	Loss: 1935.050415
Train Epoch: 7 [19200/73257 (26%)]	Loss: 1941.220581
Train Epoch: 7 [25600/73257 (35%)]	

Train Epoch: 10 [64000/73257 (87%)]	Loss: 1938.164795
Train Epoch: 10 [70400/73257 (96%)]	Loss: 1950.963013
====> Epoch: 10 Average loss: 1932.8275
Train Epoch: 11 [0/73257 (0%)]	Loss: 1907.428467
Train Epoch: 11 [6400/73257 (9%)]	Loss: 1918.861938
Train Epoch: 11 [12800/73257 (17%)]	Loss: 1961.270874
Train Epoch: 11 [19200/73257 (26%)]	Loss: 1953.478516
Train Epoch: 11 [25600/73257 (35%)]	Loss: 1931.895752
Train Epoch: 11 [32000/73257 (44%)]	Loss: 1916.414551
Train Epoch: 11 [38400/73257 (52%)]	Loss: 1926.612305
Train Epoch: 11 [44800/73257 (61%)]	Loss: 1952.878784
Train Epoch: 11 [51200/73257 (70%)]	Loss: 1901.572388
Train Epoch: 11 [57600/73257 (79%)]	Loss: 1900.313599
Train Epoch: 11 [64000/73257 (87%)]	Loss: 1934.337036
Train Epoch: 11 [70400/73257 (96%)]	Loss: 1926.945801
====> Epoch: 11 Average loss: 1932.5286
Train Epoch: 12 [0/73257 (0%)]	Loss: 1941.727905
Train Epoch: 12 [6400/73257 (9%)]	Loss: 1917.031738
Train Epoch: 12 [12800/73257 (17%)]	Loss: 1960.923706
Train Epoch: 12 

Train Epoch: 15 [51200/73257 (70%)]	Loss: 1942.942017
Train Epoch: 15 [57600/73257 (79%)]	Loss: 1943.718384
Train Epoch: 15 [64000/73257 (87%)]	Loss: 1913.895020
Train Epoch: 15 [70400/73257 (96%)]	Loss: 1896.601929
====> Epoch: 15 Average loss: 1929.2897
Train Epoch: 16 [0/73257 (0%)]	Loss: 1981.069824
Train Epoch: 16 [6400/73257 (9%)]	Loss: 1921.512817
Train Epoch: 16 [12800/73257 (17%)]	Loss: 1924.626587
Train Epoch: 16 [19200/73257 (26%)]	Loss: 1896.527954
Train Epoch: 16 [25600/73257 (35%)]	Loss: 1912.786255
Train Epoch: 16 [32000/73257 (44%)]	Loss: 1928.021118
Train Epoch: 16 [38400/73257 (52%)]	Loss: 1940.165283
Train Epoch: 16 [44800/73257 (61%)]	Loss: 1904.039185
Train Epoch: 16 [51200/73257 (70%)]	Loss: 1897.558594
Train Epoch: 16 [57600/73257 (79%)]	Loss: 1957.463135
Train Epoch: 16 [64000/73257 (87%)]	Loss: 1913.678467
Train Epoch: 16 [70400/73257 (96%)]	Loss: 1910.866333
====> Epoch: 16 Average loss: 1928.8471
Train Epoch: 17 [0/73257 (0%)]	Loss: 1916.035522
Train Epoch: 1

Train Epoch: 20 [38400/73257 (52%)]	Loss: 1930.570557
Train Epoch: 20 [44800/73257 (61%)]	Loss: 1922.014282
Train Epoch: 20 [51200/73257 (70%)]	Loss: 1910.705200
Train Epoch: 20 [57600/73257 (79%)]	Loss: 1954.176025
Train Epoch: 20 [64000/73257 (87%)]	Loss: 1915.390503
Train Epoch: 20 [70400/73257 (96%)]	Loss: 1947.262695
====> Epoch: 20 Average loss: 1927.6272
Train Epoch: 21 [0/73257 (0%)]	Loss: 1933.589355
Train Epoch: 21 [6400/73257 (9%)]	Loss: 1945.904419
Train Epoch: 21 [12800/73257 (17%)]	Loss: 1915.239136
Train Epoch: 21 [19200/73257 (26%)]	Loss: 1938.737793
Train Epoch: 21 [25600/73257 (35%)]	Loss: 1925.708496
Train Epoch: 21 [32000/73257 (44%)]	Loss: 1907.246460
Train Epoch: 21 [38400/73257 (52%)]	Loss: 1926.291382
Train Epoch: 21 [44800/73257 (61%)]	Loss: 1931.369263
Train Epoch: 21 [51200/73257 (70%)]	Loss: 1941.818848
Train Epoch: 21 [57600/73257 (79%)]	Loss: 1918.718140
Train Epoch: 21 [64000/73257 (87%)]	Loss: 1919.807251
Train Epoch: 21 [70400/73257 (96%)]	Loss: 1950.83

...

Train Epoch: 78 [64000/73257 (87%)]	Loss: 1919.486572
Train Epoch: 78 [70400/73257 (96%)]	Loss: 1906.423584
====> Epoch: 78 Average loss: 1920.0779
Train Epoch: 79 [0/73257 (0%)]	Loss: 1927.358521
Train Epoch: 79 [6400/73257 (9%)]	Loss: 1880.948364
Train Epoch: 79 [12800/73257 (17%)]	Loss: 1904.017334
Train Epoch: 79 [19200/73257 (26%)]	Loss: 1912.135620
Train Epoch: 79 [25600/73257 (35%)]	Loss: 1918.453247
Train Epoch: 79 [32000/73257 (44%)]	Loss: 1919.223267
Train Epoch: 79 [38400/73257 (52%)]	Loss: 1912.629150
Train Epoch: 79 [44800/73257 (61%)]	Loss: 1881.817139
Train Epoch: 79 [51200/73257 (70%)]	Loss: 1922.756592
Train Epoch: 79 [57600/73257 (79%)]	Loss: 1918.462402
Train Epoch: 79 [64000/73257 (87%)]	Loss: 1928.058838
Train Epoch: 79 [70400/73257 (96%)]	Loss: 1914.596924
====> Epoch: 79 Average loss: 1919.8047
Train Epoch: 80 [0/73257 (0%)]	Loss: 1916.328613
Train Epoch: 80 [6400/73257 (9%)]	Loss: 1930.859375
Train Epoch: 80 [12800/73257 (17%)]	Loss: 1916.118774
Train Epoch: 80 

Train Epoch: 83 [51200/73257 (70%)]	Loss: 1938.689697
Train Epoch: 83 [57600/73257 (79%)]	Loss: 1897.359253
Train Epoch: 83 [64000/73257 (87%)]	Loss: 1914.256714
Train Epoch: 83 [70400/73257 (96%)]	Loss: 1926.488525
====> Epoch: 83 Average loss: 1919.6995
Train Epoch: 84 [0/73257 (0%)]	Loss: 1943.270264
Train Epoch: 84 [6400/73257 (9%)]	Loss: 1922.562744
Train Epoch: 84 [12800/73257 (17%)]	Loss: 1949.403076
Train Epoch: 84 [19200/73257 (26%)]	Loss: 1919.187012
Train Epoch: 84 [25600/73257 (35%)]	Loss: 1924.031982
Train Epoch: 84 [32000/73257 (44%)]	Loss: 1932.642090
Train Epoch: 84 [38400/73257 (52%)]	Loss: 1890.284668
Train Epoch: 84 [44800/73257 (61%)]	Loss: 1909.215210
Train Epoch: 84 [51200/73257 (70%)]	Loss: 1925.468750
Train Epoch: 84 [57600/73257 (79%)]	Loss: 1912.026245
Train Epoch: 84 [64000/73257 (87%)]	Loss: 1921.661743
Train Epoch: 84 [70400/73257 (96%)]	Loss: 1933.104370
====> Epoch: 84 Average loss: 1919.6848
Train Epoch: 85 [0/73257 (0%)]	Loss: 1935.267090
Train Epoch: 8

Train Epoch: 88 [38400/73257 (52%)]	Loss: 1952.586792
Train Epoch: 88 [44800/73257 (61%)]	Loss: 1937.926514
Train Epoch: 88 [51200/73257 (70%)]	Loss: 1919.457153
Train Epoch: 88 [57600/73257 (79%)]	Loss: 1897.019287
Train Epoch: 88 [64000/73257 (87%)]	Loss: 1903.428101
Train Epoch: 88 [70400/73257 (96%)]	Loss: 1976.200195
====> Epoch: 88 Average loss: 1919.4536
Train Epoch: 89 [0/73257 (0%)]	Loss: 1913.029297
Train Epoch: 89 [6400/73257 (9%)]	Loss: 1920.430054
Train Epoch: 89 [12800/73257 (17%)]	Loss: 1915.830322
Train Epoch: 89 [19200/73257 (26%)]	Loss: 1899.370605
Train Epoch: 89 [25600/73257 (35%)]	Loss: 1920.669556
Train Epoch: 89 [32000/73257 (44%)]	Loss: 1919.602173
Train Epoch: 89 [38400/73257 (52%)]	Loss: 1934.687256
Train Epoch: 89 [44800/73257 (61%)]	Loss: 1906.135010
Train Epoch: 89 [51200/73257 (70%)]	Loss: 1894.223633
Train Epoch: 89 [57600/73257 (79%)]	Loss: 1937.530151
Train Epoch: 89 [64000/73257 (87%)]	Loss: 1906.949707
Train Epoch: 89 [70400/73257 (96%)]	Loss: 1927.99

Train Epoch: 93 [25600/73257 (35%)]	Loss: 1928.589111
Train Epoch: 93 [32000/73257 (44%)]	Loss: 1905.185547
Train Epoch: 93 [38400/73257 (52%)]	Loss: 1944.276733
Train Epoch: 93 [44800/73257 (61%)]	Loss: 1891.068115
Train Epoch: 93 [51200/73257 (70%)]	Loss: 1908.256836
Train Epoch: 93 [57600/73257 (79%)]	Loss: 1917.918091
Train Epoch: 93 [64000/73257 (87%)]	Loss: 1937.486816
Train Epoch: 93 [70400/73257 (96%)]	Loss: 1895.988892
====> Epoch: 93 Average loss: 1919.2973
Train Epoch: 94 [0/73257 (0%)]	Loss: 1930.783936
Train Epoch: 94 [6400/73257 (9%)]	Loss: 1902.012573
Train Epoch: 94 [12800/73257 (17%)]	Loss: 1884.581787
Train Epoch: 94 [19200/73257 (26%)]	Loss: 1908.731812
Train Epoch: 94 [25600/73257 (35%)]	Loss: 1881.204468
Train Epoch: 94 [32000/73257 (44%)]	Loss: 1874.842651
Train Epoch: 94 [38400/73257 (52%)]	Loss: 1957.330933
Train Epoch: 94 [44800/73257 (61%)]	Loss: 1937.842896
Train Epoch: 94 [51200/73257 (70%)]	Loss: 1914.051514
Train Epoch: 94 [57600/73257 (79%)]	Loss: 1924.17

Train Epoch: 98 [12800/73257 (17%)]	Loss: 1924.123047
Train Epoch: 98 [19200/73257 (26%)]	Loss: 1925.891846
Train Epoch: 98 [25600/73257 (35%)]	Loss: 1921.863037
Train Epoch: 98 [32000/73257 (44%)]	Loss: 1907.731812
Train Epoch: 98 [38400/73257 (52%)]	Loss: 1905.110229
Train Epoch: 98 [44800/73257 (61%)]	Loss: 1943.954956
Train Epoch: 98 [51200/73257 (70%)]	Loss: 1922.945190
Train Epoch: 98 [57600/73257 (79%)]	Loss: 1915.965454
Train Epoch: 98 [64000/73257 (87%)]	Loss: 1880.881958
Train Epoch: 98 [70400/73257 (96%)]	Loss: 1927.752197
====> Epoch: 98 Average loss: 1919.0607
Train Epoch: 99 [0/73257 (0%)]	Loss: 1928.420288
Train Epoch: 99 [6400/73257 (9%)]	Loss: 1905.017578
Train Epoch: 99 [12800/73257 (17%)]	Loss: 1928.377441
Train Epoch: 99 [19200/73257 (26%)]	Loss: 1949.980469
Train Epoch: 99 [25600/73257 (35%)]	Loss: 1938.182617
Train Epoch: 99 [32000/73257 (44%)]	Loss: 1929.999390
Train Epoch: 99 [38400/73257 (52%)]	Loss: 1890.117920
Train Epoch: 99 [44800/73257 (61%)]	Loss: 1923.37

In [0]:
with torch.no_grad():
        sample = torch.randn(64, 32).cuda()
        sample = model.decode(sample)
        torch.save(model.cpu().state_dict(), "svhn_save_checkpoint_epoch_"+str(epoch)+".pth")
        files.download("svhn_save_checkpoint_epoch_"+str(epoch)+".pth")
        torchvision.utils.save_image(sample.view(64, 3, 32, 32),'sample_svhn_' + str(epoch) + '.png')
        files.download('sample_svhn_' + str(epoch) + '.png')